In [1]:
import torch
import os
import requests
import cv2
import glob
import numpy as np
from collections import OrderedDict

from PIL import Image

In [ ]:
from SwinIR.network_swinir import SwinIR as net

In [ ]:
from SwinIR.util_calculate_psnr_ssim import calculate_psnr, calculate_ssim, bgr2ycbcr

In [15]:
models = [
'001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth', # training patch 64 and scale 2
'001_classicalSR_DF2K_s64w8_SwinIR-M_x3.pth',
'001_classicalSR_DF2K_s64w8_SwinIR-M_x4.pth',
'001_classicalSR_DF2K_s64w8_SwinIR-M_x8.pth', # training patch 64 and scale 8
'001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth',# training patch 48 and scale 2
'001_classicalSR_DIV2K_s48w8_SwinIR-M_x3.pth',
'001_classicalSR_DIV2K_s48w8_SwinIR-M_x4.pth',
'001_classicalSR_DIV2K_s48w8_SwinIR-M_x8.pth',# training patch 48 and scale 8
'002_lightweightSR_DIV2K_s64w8_SwinIR-S_x2.pth',# training patch 64 and scale 2
'002_lightweightSR_DIV2K_s64w8_SwinIR-S_x3.pth',
'002_lightweightSR_DIV2K_s64w8_SwinIR-S_x4.pth',# training patch 64 and scale 4
'003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth',
'003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_PSNR-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_PSNR.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_PSNR-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_PSNR.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_PSNR-with-dict-keys-params-and-params_ema.pth',
'003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_PSNR.pth',
'004_grayDN_DFWB_s128w8_SwinIR-M_noise15.pth',
'004_grayDN_DFWB_s128w8_SwinIR-M_noise25.pth',
'004_grayDN_DFWB_s128w8_SwinIR-M_noise50.pth',
'005_colorDN_DFWB_s128w8_SwinIR-M_noise15.pth',
'005_colorDN_DFWB_s128w8_SwinIR-M_noise25.pth',
'005_colorDN_DFWB_s128w8_SwinIR-M_noise50.pth',
'006_CAR_DFWB_s126w7_SwinIR-M_jpeg10.pth',
'006_CAR_DFWB_s126w7_SwinIR-M_jpeg20.pth',
'006_CAR_DFWB_s126w7_SwinIR-M_jpeg30.pth',
'006_CAR_DFWB_s126w7_SwinIR-M_jpeg40.pth',
'006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg10.pth',
'006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg20.pth',
'006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg30.pth',
'006_colorCAR_DFWB_s126w7_SwinIR-M_jpeg40.pth']

In [21]:
def download_model(model_name):    
    model_path = '/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/models/'
    if os.path.exists(model_path + model_name):
        print(f'Model {model_name} already exists at {model_path}.')
    else:
        print(f'Model {model_name} not found. Downloading...')
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/{}'.format(model_name)
        print(f'downloading model {url}')
        r = requests.get(url, allow_redirects=True)
        open(model_path+model_name, 'wb').write(r.content)
        print(f'Downloaded model {model_name} to {model_path}')    

In [22]:
download_model(models[0])

Model 001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth not found. Downloading...
downloading model https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth
Downloaded model 001_classicalSR_DF2K_s64w8_SwinIR-M_x2.pth to /Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/models/


In [26]:
def define_model(task, scale, training_patch, large_model, model_name) -> net:
    model_path = '/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/models/' + model_name
    # 001 classical image sr
    if task == 'classical_sr':
        model = net(upscale=scale, in_chans=3, img_size=training_patch, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='pixelshuffle', resi_connection='1conv')
        param_key_g = 'params'

    # 002 lightweight image sr
    # use 'pixelshuffledirect' to save parameters
    elif task == 'lightweight_sr':
        model = net(upscale=scale, in_chans=3, img_size=64, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6], embed_dim=60, num_heads=[6, 6, 6, 6],
                    mlp_ratio=2, upsampler='pixelshuffledirect', resi_connection='1conv')
        param_key_g = 'params'

    # 003 real-world image sr
    elif task == 'real_sr':
        if not large_model:
            # use 'nearest+conv' to avoid block artifacts
            model = net(upscale=scale, in_chans=3, img_size=64, window_size=8,
                        img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                        mlp_ratio=2, upsampler='nearest+conv', resi_connection='1conv')
        else:
            # larger model size; use '3conv' to save parameters and memory; use ema for GAN training
            model = net(upscale=scale, in_chans=3, img_size=64, window_size=8,
                        img_range=1., depths=[6, 6, 6, 6, 6, 6, 6, 6, 6], embed_dim=240,
                        num_heads=[8, 8, 8, 8, 8, 8, 8, 8, 8],
                        mlp_ratio=2, upsampler='nearest+conv', resi_connection='3conv')
        param_key_g = 'params_ema'

    # 004 grayscale image denoising
    elif task == 'gray_dn':
        model = net(upscale=1, in_chans=1, img_size=128, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 005 color image denoising
    elif task == 'color_dn':
        model = net(upscale=1, in_chans=3, img_size=128, window_size=8,
                    img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 006 grayscale JPEG compression artifact reduction
    # use window_size=7 because JPEG encoding uses 8x8; use img_range=255 because it's sligtly better than 1
    elif task == 'jpeg_car':
        model = net(upscale=1, in_chans=1, img_size=126, window_size=7,
                    img_range=255., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    # 006 color JPEG compression artifact reduction
    # use window_size=7 because JPEG encoding uses 8x8; use img_range=255 because it's sligtly better than 1
    elif task == 'color_jpeg_car':
        model = net(upscale=1, in_chans=3, img_size=126, window_size=7,
                    img_range=255., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                    mlp_ratio=2, upsampler='', resi_connection='1conv')
        param_key_g = 'params'

    pretrained_model = torch.load(model_path)
    model.load_state_dict(pretrained_model[param_key_g] if param_key_g in pretrained_model.keys() else pretrained_model, strict=True)

    return model

In [33]:
upscale = 2
window_size = 8
height = (1024 // upscale // window_size + 1) * window_size
width = (720 // upscale // window_size + 1) * window_size
model = net(upscale=2, img_size=(height, width),
                   window_size=window_size, img_range=1., depths=[6, 6, 6, 6],
                   embed_dim=60, num_heads=[6, 6, 6, 6], mlp_ratio=2, upsampler='pixelshuffledirect')
#print(model)
print(height, width, model.flops() / 1e9)

x = torch.randn((1, 3, height, width))
print(f"Tensor shape: {x.shape}")
x = model(x)
print(f"Output shape: {x.shape}")

520 368 195.589056
Tensor shape: torch.Size([1, 3, 520, 368])
Output shape: torch.Size([1, 3, 1040, 736])


In [34]:
def setup_folder(task, scale, folder_gt, large_model=False):
    save_dir = f"/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/images/{task}/x{scale}"
    folder = folder_gt
    if task in ['classical_sr', 'lightweight_sr']:  
        border = scale
        window_size = 8
    
    elif task in ['real_sr']:
        if large_model:
            save_dir += '_large'
        border = 0
        window_size = 8
    
    elif task in ['gray_dn', 'color_dn']:
        border = 0
        window_size = 8
        
    elif task in ['jpeg_car', 'color_jpeg_car']:
        border = 0
        window_size = 7
        
    return folder, save_dir, border, window_size

In [37]:
def get_image_pair(task, path, folder_lq, scale, noise=15, jpeg=40):
    (imgname, imgext) = os.path.splitext(os.path.basename(path))
    
    # 001 classical image sr/ 002 lightweight image sr (load lq-gt image pairs)
    if task in ['classical_sr', 'lightweight_sr']:
        img_gt = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
        img_lq = cv2.imread(f'{folder_lq}/{imgname}x{scale}{imgext}', cv2.IMREAD_COLOR).astype(
            np.float32) / 255.

    # 003 real-world image sr (load lq image only)
    elif task in ['real_sr']:
        img_gt = None
        img_lq = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.

    # 004 grayscale image denoising (load gt image and generate lq image on-the-fly)
    elif task in ['gray_dn']:
        img_gt = cv2.imread(path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.
        np.random.seed(seed=0)
        img_lq = img_gt + np.random.normal(0, noise / 255., img_gt.shape)
        img_gt = np.expand_dims(img_gt, axis=2)
        img_lq = np.expand_dims(img_lq, axis=2)

    # 005 color image denoising (load gt image and generate lq image on-the-fly)
    elif task in ['color_dn']:
        img_gt = cv2.imread(path, cv2.IMREAD_COLOR).astype(np.float32) / 255.
        np.random.seed(seed=0)
        img_lq = img_gt + np.random.normal(0, noise / 255., img_gt.shape)

    # 006 grayscale JPEG compression artifact reduction (load gt image and generate lq image on-the-fly)
    elif task in ['jpeg_car']:
        img_gt = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img_gt.ndim != 2:
            img_gt = bgr2ycbcr(img_gt, y_only=True)
        result, encimg = cv2.imencode('.jpg', img_gt, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg])
        img_lq = cv2.imdecode(encimg, 0)
        img_gt = np.expand_dims(img_gt, axis=2).astype(np.float32) / 255.
        img_lq = np.expand_dims(img_lq, axis=2).astype(np.float32) / 255.

    # 006 JPEG compression artifact reduction (load gt image and generate lq image on-the-fly)
    elif task in ['color_jpeg_car']:
        img_gt = cv2.imread(path)
        result, encimg = cv2.imencode('.jpg', img_gt, [int(cv2.IMWRITE_JPEG_QUALITY), jpeg])
        img_lq = cv2.imdecode(encimg, 1)
        img_gt = img_gt.astype(np.float32)/ 255.
        img_lq = img_lq.astype(np.float32)/ 255.

    return imgname, img_lq, img_gt   

In [36]:
def test(img_lq, model, args, window_size):
    if args.tile is None:
        # test the image as a whole
        output = model(img_lq)
    else:
        # test the image tile by tile
        b, c, h, w = img_lq.size()
        tile = min(args.tile, h, w)
        assert tile % window_size == 0, "tile size should be a multiple of window_size"
        tile_overlap = args.tile_overlap
        sf = args.scale

        stride = tile - tile_overlap
        h_idx_list = list(range(0, h-tile, stride)) + [h-tile]
        w_idx_list = list(range(0, w-tile, stride)) + [w-tile]
        E = torch.zeros(b, c, h*sf, w*sf).type_as(img_lq)
        W = torch.zeros_like(E)

        for h_idx in h_idx_list:
            for w_idx in w_idx_list:
                in_patch = img_lq[..., h_idx:h_idx+tile, w_idx:w_idx+tile]
                out_patch = model(in_patch)
                out_patch_mask = torch.ones_like(out_patch)

                E[..., h_idx*sf:(h_idx+tile)*sf, w_idx*sf:(w_idx+tile)*sf].add_(out_patch)
                W[..., h_idx*sf:(h_idx+tile)*sf, w_idx*sf:(w_idx+tile)*sf].add_(out_patch_mask)
        output = E.div_(W)

    return output

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

In [46]:
model = define_model('classical_sr', 
                     2, 
                     64, 
                     False, 
                     models[0])

In [47]:
model.eval()
model.to(device)

SwinIR(
  (conv_first): Conv2d(3, 180, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=180, input_resolution=(64, 64), depth=6
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=180, input_resolution=(64, 64), num_heads=6, window_size=8, shift_size=0, mlp_ratio=2
            (norm1): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=180, window_size=(8, 8), num_heads=6
              (qkv): Linear(in_features=180, out_features=540, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=180, out_features=180, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
       

In [57]:
# Load the input image
input_path = 'SwinIR/images/X2/butterflyx2.png'
img = cv2.imread(input_path, cv2.IMREAD_COLOR).astype(np.float32) / 255.0
img = np.transpose(img if img.shape[2] == 1 else img[:, :, [2, 1, 0]], (2, 0, 1))

# Convert to tensor and add batch dimension
img_tensor = torch.from_numpy(img).unsqueeze(0).to(device)
window_size = 8  # Assuming window size is 8 for the model
# Run through model
with torch.no_grad():   
    _, _, h_old, w_old = img_tensor.shape
    h_pad = (h_old // window_size + 1) * window_size - h_old
    w_pad = (w_old // window_size + 1) * window_size - w_old
    image = torch.cat([img_tensor, torch.flip(img_tensor, [2])], 2)[:, :, :h_old + h_pad, :]
    image = torch.cat([image, torch.flip(image, [3])], 3)[:, :, :, :w_old + w_pad]
            
    output = model(image)
    output = output[:, :, :h_old * 2, :w_old * 2]  # Crop to original size

# Convert output tensor to image
output_img = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
if output_img.ndim == 3:
    output_img = np.transpose(output_img[[2,1,0], :, :], (1,2,0) )   
output_img = (output_img * 255.0).round().astype(np.uint8)

# Save the output image
cv2.imwrite('SwinIR/images/X2/butterflyx2_output.png', output_img)

True

In [61]:
def evaluate_image_psnr_ssim(img_1, img_2):
    """
    Evaluate PSNR and SSIM between two images.
    
    Args:
        img_1 (numpy.ndarray): First image.
        img_2 (numpy.ndarray): Second image.
        
    Returns:
        tuple: PSNR and SSIM values.
    """
    psnr = calculate_psnr(img_1, img_2, crop_border=8)
    ssim = calculate_ssim(img_1, img_2, crop_border=8)
    return psnr, ssim
    

In [ ]:
img_1 = output_img
img_2 = cv2.imread('/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/images/HR/butterfly.png', cv2.IMREAD_COLOR).astype(np.float32) / 255.0
img_gt = (img_2 * 255.0).round().astype(np.uint8)  # float32 to uint8
img_gt = img_gt[:h_old * 2, :w_old * 2, ...]  # crop gt
img_gt = np.squeeze(img_gt)

In [63]:
eval_psnr, eval_ssim = evaluate_image_psnr_ssim(img_1, img_gt)
print(f"PSNR: {eval_psnr:.2f}, SSIM: {eval_ssim:.4f}")

PSNR: 34.70, SSIM: 0.9739


In [ ]:
folder, save_dir, border, window_size = setup_folder(
    'classical_sr', 
    2, 
    '/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/images/classical_sr/x2', large_model=False)

In [ ]:
# Example: Fine-tuning the SwinIR model on a single image (img_tensor, img_gt)
# Note: This is a minimal example for demonstration. For real fine-tuning, use a dataset and DataLoader.

import torch.optim as optim
import torch.nn as nn

# Prepare data
input_tensor = img_tensor  # shape: (1, 3, H, W)
target = torch.from_numpy(np.transpose(img_gt, (2, 0, 1))).unsqueeze(0).float().to(device) / 255.0

# Set model to train mode
model.train()

# Define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.L1Loss()

# Fine-tune for a few steps
num_steps = 5
for step in range(num_steps):
    optimizer.zero_grad()
    output = model(input_tensor)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if (step + 1) % 10 == 0:
        print(f"Step {step+1}/{num_steps}, Loss: {loss.item():.6f}")

# Set back to eval mode after fine-tuning
model.eval()

In [3]:
# Prepare dataset: X2 folder as input (low-resolution), HR folder as target (high-resolution)
input_folder = '/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/images/X2'
target_folder = '/Volumes/TOSHIBA/Github Repositories/transformers/SwinIR/images/HR'

input_images = sorted(glob.glob(os.path.join(input_folder, '*.png')))
target_images = sorted(glob.glob(os.path.join(target_folder, '*.png')))

print(f"Number of input images: {len(input_images)}")
print(f"Number of target images: {len(target_images)}")

Number of input images: 10
Number of target images: 5


In [ ]:
import torch.nn as nn

# Prepare data
input_tensor = img_tensor  # shape: (1, 3, H, W)
target = torch.from_numpy(np.transpose(img_gt, (2, 0, 1))).unsqueeze(0).float().to(device) / 255.0

# Set model to train mode
model.train()

# Define optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

# Fine-tune for a few steps
num_steps = 5
for step in range(num_steps):
    optimizer.zero_grad()
    output = model(input_tensor)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if (step + 1) % 10 == 0:
        print(f"Step {step+1}/{num_steps}, Loss: {loss.item():.6f}")

# Set back to eval mode after fine-tuning
model.eval()

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImagePairDataset(Dataset):
    def __init__(self, input_paths, target_paths, transform=None):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.transform = transform

    def __len__(self):
        return len(self.input_paths)

    def __getitem__(self, idx):
        # Load input (low-res) image
        img_input = cv2.imread(self.input_paths[idx], cv2.IMREAD_COLOR).astype(np.float32) / 255.0
        img_input = np.transpose(img_input[:, :, [2, 1, 0]], (2, 0, 1))  # BGR to RGB, HWC to CHW

        # Load target (high-res) image
        img_target = cv2.imread(self.target_paths[idx], cv2.IMREAD_COLOR).astype(np.float32) / 255.0
        img_target = np.transpose(img_target[:, :, [2, 1, 0]], (2, 0, 1))

        input_tensor = torch.from_numpy(img_input).float()
        target_tensor = torch.from_numpy(img_target).float()

        if self.transform:
            input_tensor, target_tensor = self.transform(input_tensor, target_tensor)

        return input_tensor, target_tensor

# Create dataset and dataloader
dataset = ImagePairDataset(input_images, target_images)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [ ]:
# Fine-tune the model using the dataloader with batch size 2

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

num_steps = 5
step = 0
for input_batch, target_batch in dataloader:
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    optimizer.step()
    step += 1
    if step >= num_steps:
        break

model.eval()

In [ ]:
# Evaluate the fine-tuned model on the dataset
model.eval()
psnr_list = []
ssim_list = []

for inp_path, tgt_path in zip(input_images, target_images):
    # Load input and target images
    inp_img = cv2.imread(inp_path, cv2.IMREAD_COLOR).astype(np.float32) / 255.0
    tgt_img = cv2.imread(tgt_path, cv2.IMREAD_COLOR).astype(np.float32) / 255.0

    # Prepare input tensor
    inp_tensor = np.transpose(inp_img[:, :, [2, 1, 0]], (2, 0, 1))
    inp_tensor = torch.from_numpy(inp_tensor).unsqueeze(0).to(device)

    # Pad input if needed
    _, _, h_old, w_old = inp_tensor.shape
    h_pad = (h_old // 8 + 1) * 8 - h_old
    w_pad = (w_old // 8 + 1) * 8 - w_old
    image = torch.cat([inp_tensor, torch.flip(inp_tensor, [2])], 2)[:, :, :h_old + h_pad, :]
    image = torch.cat([image, torch.flip(image, [3])], 3)[:, :, :, :w_old + w_pad]

    # Inference
    with torch.no_grad():
        output = model(image)
        output = output[:, :, :h_old * 2, :w_old * 2]

    # Convert output tensor to image
    output_img = output.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    if output_img.ndim == 3:
        output_img = np.transpose(output_img[[2, 1, 0], :, :], (1, 2, 0))
    output_img = (output_img * 255.0).round().astype(np.uint8)

    # Prepare ground truth
    tgt_img_uint8 = (tgt_img * 255.0).round().astype(np.uint8)
    tgt_img_uint8 = tgt_img_uint8[:h_old * 2, :w_old * 2, ...]
    tgt_img_uint8 = np.squeeze(tgt_img_uint8)

    # Evaluate
    psnr, ssim = evaluate_image_psnr_ssim(output_img, tgt_img_uint8)
    psnr_list.append(psnr)
    ssim_list.append(ssim)

print(f"Average PSNR: {np.mean(psnr_list):.2f}, Average SSIM: {np.mean(ssim_list):.4f}")